# A full training

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [7]:
!pip install dataset evaluate transformers[sentencepiece] 
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

ERROR: Could not find a version that satisfies the requirement evaluate (from versions: none)
ERROR: No matching distribution found for evaluate


In [35]:
!pip install evaluate

ERROR: Could not find a version that satisfies the requirement evaluate (from versions: none)
ERROR: No matching distribution found for evaluate


In [12]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 4.0 MB 27.1 MB/s            
     |████████████████████████████████| 880 kB 107.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.6 MB 95.2 MB/s            
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=0eedc8a6eb0bf2ba25a9f6dd3080c1ed4e26036b392a843f1cca891fa18035a1
  Stored in directory: /root/.cache/pip/wheels/4c/64/31/e9900a234b23fb3e9dc565d6114a9d6ff84a72dbdd356502b4
Successfully built sacremoses


In [14]:
!pip install  transformers[sentencepiece]

     |████████████████████████████████| 1.3 MB 26.6 MB/s            


In [15]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [19]:
!git clone https://github.com/cryptexcode/nlp_hackathon_bd_2023

Cloning into 'nlp_hackathon_bd_2023'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 27 (delta 5), reused 22 (delta 4), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [20]:
trainDataDir= "nlp_hackathon_bd_2023/data/train.txt"
devDataDir= "nlp_hackathon_bd_2023/data/dev.txt"

In [21]:
df = pd.read_csv(trainDataDir, names=["text","label"], sep=" _ _ ", header=None)
dfVal=pd.read_csv(devDataDir, names=["text","label"], sep=" _ _ ", header=None)

df = df[:1000]
dfVal = dfVal[:1000]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [22]:
df

,text,label
0,তার,O
1,মৃত্যুর,O
2,দশ,O
3,দিন,O
4,"পর,",O
...,...,...
995,বেশি,O
996,ক্ষুদ্র,B-CORP
997,ও,I-CORP
998,মাঝারি,I-CORP


In [23]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

  


In [24]:
df["label"]=label_encoder.fit_transform(df["label"]);
dfVal["label"]=label_encoder.transform(dfVal["label"]);

In [25]:
datasetTrain= Dataset.from_pandas(df)
datasetVal= Dataset.from_pandas(dfVal)

In [26]:



checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = datasetTrain.map(tokenize_function, batched=True)
tokenized_datasetVal= datasetVal.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [27]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets.column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [28]:
tokenized_datasetVal = tokenized_datasetVal.remove_columns(["text"])
tokenized_datasetVal = tokenized_datasetVal.rename_column("label", "labels")
tokenized_datasetVal.set_format("torch")
tokenized_datasetVal.column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [29]:
["attention_mask", "input_ids", "labels", "token_type_ids"]

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [30]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets, shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasetVal, batch_size=8, collate_fn=data_collator
)

In [31]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

[2023-01-21 07:32:18.996 pytorch-1-8-gpu-py3-ml-g4dn-xlarge-60bd0d07a83be181dcf7335baae2:460 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-01-21 07:32:19.087 pytorch-1-8-gpu-py3-ml-g4dn-xlarge-60bd0d07a83be181dcf7335baae2:460 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 10]),
 'token_type_ids': torch.Size([8, 10]),
 'attention_mask': torch.Size([8, 10])}

In [32]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=13)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [33]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(2.5895, grad_fn=<NllLossBackward>) torch.Size([8, 13])


In [34]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/opt/conda/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [37]:
from transformers import get_scheduler

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

250


In [38]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [39]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/250 [00:00<?, ?it/s]

In [40]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(training_function)